In [24]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib
import re
import gget
import requests
import warnings

In [25]:
os.getcwd()

'c:\\Users\\samue\\OneDrive\\Documenti\\Studio\\Università\\Tesi\\Progetto\\Ricerca\\Symbol\\Annotated'

In [9]:
os.chdir("..")

In [10]:
path = "/SYMBOL"

In [ ]:
os.chdir(".."+path)

<h3>Lettura dei dataset normalizzati con RMA e filtrati</h3>

In [14]:
os.chdir('Annotated')

In [26]:
gse41848 = pd.read_csv('GSE41848_filtered.csv', index_col=0)
gse41849 = pd.read_csv('GSE41849_filtered.csv', index_col=0)
gse146383 = pd.read_csv('GSE146383_filtered.csv', index_col=0)
gse13732 = pd.read_csv('GSE13732_filtered.csv', index_col=0)
gse136411 = pd.read_csv('GSE136411_filtered.csv', index_col=0)
gse17048 = pd.read_csv('GSE17048_filtered.csv', index_col=0)
gse41890 = pd.read_csv('GSE41890_filtered.csv', index_col=0)
gse21942 = pd.read_csv('GSE21942_filtered.csv', index_col=0)

In [27]:
# datasetList = [gse41848, gse41849, gse146383, gse13732, gse136411, gse17048, gse41890, gse21942]
# nameList = ['GSE41848', 'GSE41849', 'GSE146383', 'GSE13732', 'GSE136411', 'GSE17048', 'GSE41890', 'GSE21942']

datasetList = [gse41848, gse41849, gse146383, gse13732, gse17048, gse41890, gse21942]
nameList = ['GSE41848', 'GSE41849', 'GSE146383', 'GSE13732', 'GSE17048', 'GSE41890', 'GSE21942']

<h3>Drop del genere</h3> 

In [28]:
for dataset, datasetName in zip(datasetList, nameList):
    if 'Gender' in dataset.columns:
        print(datasetName)
        dataset.drop(['Gender'], axis=1, inplace=True)

GSE41849


<h3>Calcolo dei geni comuni ai vari dataset</h3>

In [29]:
geneList = [dataset.columns[1:] for dataset in datasetList]

def corrispondenze(arrays, n):
    print(len(arrays))
    corrispondenzeMultiple = set()
    corrispondenzeEsatte = set()
    conteggio = {}
    
    for array in arrays:
        for stringa in set(array):
            conteggio[stringa] = conteggio.get(stringa, 0) + 1
    
    for stringa, conteggio in conteggio.items():
        if conteggio >= n-1:
            corrispondenzeMultiple.add(stringa)
        if conteggio == n:
            corrispondenzeEsatte.add(stringa)
    
    return list(corrispondenzeEsatte), list(corrispondenzeMultiple)

corrispondenzeEsatte, corrispondenzeMultiple = corrispondenze(geneList, len(geneList))

print(len(corrispondenzeEsatte), len(corrispondenzeMultiple))
# print("Geni in comune a GSE41848:", len(set(corrispondenzeMultiple) & set(geneList[0])))
# print("Geni in comune a GSE41849:", len(set(corrispondenzeMultiple) & set(geneList[1])))
# print("Geni in comune a GSE146383:", len(set(corrispondenzeMultiple) & set(geneList[2])))
# print("Geni in comune a GSE13732:", len(set(corrispondenzeMultiple) & set(geneList[3])))
# print("Geni in comune a GSE136411:", len(set(corrispondenzeMultiple) & set(geneList[4])))
# print("Geni in comune a GSE17048:", len(set(corrispondenzeMultiple) & set(geneList[5])))
# print("Geni in comune a GSE41890:", len(set(corrispondenzeMultiple) & set(geneList[6])))
# print("Geni in comune a GSE21942:", len(set(corrispondenzeMultiple) & set(geneList[7])))

7
11492 15969


<h3>Merging dei vari dataset in un'unico dataset sulla base dei geni in comune</h3>

In [30]:
def add(dataset, columns):
    for column in columns:
        if column not in dataset.columns.values:
            print("aggiungo")
            dataset[column] = 0
    return dataset


def merge(datasets, genes):
    merged = pd.DataFrame()
    columns = ['SampleID']+genes
    length = 0
    for dataset in datasets:
        if ~ np.all(np.isin(dataset.columns.values, columns)): dataset=add(dataset, columns)
        filtered = dataset[columns]
        if merged.empty:
            merged = filtered
        else:
            merged = pd.concat([merged, filtered], ignore_index=True)
    
    return merged

mergedDataset = merge(datasetList, genes=corrispondenzeEsatte)

def rename(entry, sampleSeries):
    for i in range(0, len(sampleSeries)):
        if entry in sampleSeries[i].values:
            return str(i)+'-'+entry

mergedDataset['SampleID'] = mergedDataset['SampleID'].apply(lambda x: rename(x, [dataset['SampleID'] for dataset in datasetList]))

def etichetta(entry):
    return 0 if 'Control' in entry else 1

label = pd.DataFrame()
label['SampleID'] = mergedDataset['SampleID']
label['Label'] = mergedDataset['SampleID'].apply(lambda x: etichetta(x))

mergedDataset = label.merge(mergedDataset, on='SampleID', how='inner')

print(mergedDataset.shape)
print("I pazienti malati sono:", mergedDataset['Label'].sum())
print("I pazienti sani sono:", mergedDataset.shape[0] - mergedDataset['Label'].sum())
mergedDataset

(729, 11494)
I pazienti malati sono: 434
I pazienti sani sono: 295


,SampleID,Label,XPO6,ARPC1A,THOP1,ATP2B2,CSTA,PKM,MAPK1IP1L,ABCG2,...,GPR25,LRRC31,GPR88,LAMTOR3,ACTL6A,MS4A6A,BAZ2A,TPM1,DIABLO,DOC2A
0,0-GSM1026056_600009.0001,1,11.650549,5.956854,7.162408,6.453192,6.417070,9.921520,9.345639,5.526452,...,7.967350,3.591659,6.990793,8.102131,6.348476,8.500851,8.825179,7.069051,7.250345,6.006205
1,0-GSM1026057_600009.0001-FollowUp_1,1,11.868237,5.697745,7.424835,6.495434,6.685321,10.243414,9.292087,5.236324,...,8.134902,3.375409,6.983589,8.120959,6.256685,8.546182,9.181231,6.840454,7.261851,6.411964
2,0-GSM1026058_41461.0001,1,12.204048,6.588301,7.326014,6.598216,6.348511,9.966085,9.231655,5.071786,...,8.168079,3.773682,7.172718,7.942987,6.254222,8.844696,9.155631,7.067990,6.993573,6.368026
3,0-GSM1026059_41462.0001,1,12.176452,6.606812,7.162027,6.207555,7.208305,9.979140,9.465428,5.545075,...,7.850931,3.516395,6.631965,8.697892,6.409212,9.180507,9.190014,7.263440,7.425670,5.655392
4,0-GSM1026060_600029.0001,1,12.151468,6.195992,7.083782,6.361748,7.705152,10.225707,9.573062,5.356958,...,7.915110,3.280334,6.707222,8.457575,6.421640,9.398804,9.362108,6.928783,7.118576,5.806177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,6-GSM545842_M10,1,9.325217,8.476524,6.215869,5.096182,11.247372,10.479694,10.073199,3.730077,...,3.598088,2.912370,2.874592,9.129791,7.374601,10.937702,7.659700,8.224351,8.772679,4.732198
725,6-GSM545843_M11,1,9.240105,8.279137,5.773038,4.877793,11.246373,9.645089,10.210183,3.555146,...,3.375732,3.044542,2.536759,9.228510,8.381170,10.820722,7.137152,7.823008,8.878464,5.040987
726,6-GSM545844_M11-FollowUp_1,1,9.348508,7.925483,5.735498,4.853527,11.318210,9.527473,10.650822,3.820406,...,3.153846,2.919875,2.604844,9.268521,8.231973,10.698401,7.404353,8.017363,8.946480,5.257213
727,6-GSM545845_M12,1,9.150152,8.293933,5.593354,4.877793,11.183782,9.740226,10.254200,4.051920,...,3.460705,3.024201,2.660720,9.390753,8.217098,10.725866,7.098828,8.318048,8.803759,4.973154


Controllo 0 e NA

In [31]:
print(mergedDataset.columns.isna().sum())
print(mergedDataset.isna().sum().sum())
print(mergedDataset[mergedDataset.columns[2:]].eq(0).sum().sum())

0
0
0


<h3>Salvataggio del dataset Merged</h3>

In [22]:
os.getcwd()

'c:\\Users\\samue\\OneDrive\\Documenti\\Studio\\Università\\Tesi\\Progetto\\Ricerca\\Symbol\\Annotated'

In [32]:
mergedDataset.to_csv('../Dataset/MergedDatasetReduced_symbol.csv', index=False)

<h3>Mapping Ensembl to Symbol<h3>

In [17]:
def ensembl_to_symbol(ensembl_id):
    url = f"https://rest.ensembl.org/lookup/id/{ensembl_id}?content-type=application/json"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data.get("display_name", None)
    return None

newColumns = [ensembl_to_symbol(gene) for gene in mergedDataset.columns[2:]]